## Imports

In [1]:
import pandas as pd
from tqdm.notebook import tqdm
from pprint import pprint
import numpy as np
from pymongo import MongoClient

## Connect to mongo database

In [2]:
# Connect to Mongo server
host="rhea.isegi.unl.pt"
port="27049"
user="GROUP_32"
password="bRG2XjRZhrRA9IfpmENyXxMlWQDUJdzL"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

# Connect to mongo db
db = client.denormalised

In [6]:
db.list_collection_names()

['Application_Countries',
 'Warehouse_Colors',
 'Sales_OrderLines',
 'Warehouse_StockItemTransactions',
 'Application_StateProvinces',
 'Sales_Invoices',
 'Sales_Orders',
 'Purchasing_PurchaseOrders',
 'Application_DeliveryMethods',
 'Application_Cities',
 'Application_TransactionTypes',
 'Warehouse_StockItems',
 'Sales_InvoiceLines',
 'Warehouse_StockItemStockGroups',
 'Purchasing_PurchaseOrderLines',
 'Purchasing_SupplierCategories',
 'Application_PaymentMethods',
 'Sales_Customers',
 'Sales_CustomerCategories',
 'Purchasing_SupplierTransactions',
 'Application_People',
 'Warehouse_PackageTypes',
 'Purchasing_Suppliers',
 'reviews',
 'Warehouse_StockGroups',
 'Sales_CustomerTransactions']

In [4]:
pprint(db['Application_Cities'].find_one())

{'CityID': 1,
 'CityName': 'Aaronsburg',
 'LatestRecordedPopulation': 613,
 'Location': '0xE6100000010C07E11B542C73444087C09140035D53C0',
 'StateProvince': {'Country': {'Continent': 'North America',
                               'CountryID': 230,
                               'CountryName': 'United States',
                               'CountryType': 'UN Member State',
                               'FormalName': 'United States of America',
                               'IsoAlpha3Code': 'USA',
                               'IsoNumericCode': 840,
                               'LatestRecordedPopulation': 313973000,
                               'Region': 'Americas',
                               'Subregion': 'Northern America'},
                   'CountryID': 230,
                   'LatestRecordedPopulation': 13284753,
                   'SalesTerritory': 'Mideast',
                   'StateProvinceCode': 'PA',
                   'StateProvinceID': 39,
                   'Stat

The warehouse group needs:  
- Which items get ordered the most in bulk (bigger amounts)? 

In [5]:
query_2={
    '$group': {
        '_id': {'Description' : '$Description'},
        'average_quantity' : {'$avg' : '$Quantity'}
    }
}

query_5 = {
    '$sort' : { 'average_quantity' : -1 }
}

query_6 = {
    '$limit': 3}

query_7 = {
    '$project': {
        '_id' : False,
        'name' : '$_id.Description',
        'Quantity': '$average_quantity'}
}

pipeline = [query_2, query_5, query_6,query_7]

r = db.Sales_OrderLines.aggregate(pipeline)

result = list(r)

result

[{'name': 'Black and orange fragile despatch tape 48mmx75m',
  'Quantity': 198.8181818181818},
 {'name': 'Black and orange fragile despatch tape 48mmx100m',
  'Quantity': 197.26182965299685},
 {'name': 'Shipping carton (Brown) 356x356x279mm',
  'Quantity': 146.47058823529412}]

The warehouse group needs:  
- Which customers have delivery addresses under 10km of distance?  

In [7]:
query_1={
    '$project': {
        '_id': False,
        'CostumerID' : '$CustomerID',
        'location' : {'lat':'$DeliveryLocationLat', 'long':'$DeliveryLocationLong' }
    }
}

query_2={
    '$find': 
   {
     '$location':
       { '$near':
          {
            '$geometry': {'location': [ 41.4972, -102.62 ] },
            '$minDistance': 0,
            '$maxDistance': 10000
          }
       }
   }
}
pipeline = [query_1,query_2]

r = db.Sales_Customers.aggregate(pipeline)


result = list(r)

result

OperationFailure: Unrecognized pipeline stage name: '$find', full error: {'ok': 0.0, 'errmsg': "Unrecognized pipeline stage name: '$find'", 'code': 40324, 'codeName': 'Location40324'}

https://docs.mongodb.com/manual/geospatial-queries/#example